## Read data

In [1]:
import pandas as pd

In [2]:
path1 = r'C:\Users\hahas\Desktop\Marketing Analytics\Final Case\customer_service_reps'
path2 = r'C:\Users\hahas\Desktop\Marketing Analytics\Final Case\engagement'
path3 = r'C:\Users\hahas\Desktop\Marketing Analytics\Final Case\subscribers'
path4 = r'C:\Users\hahas\Desktop\Marketing Analytics\Final Case\advertising_spend_data.xlsx'

customer = pd.read_pickle(path1)
engagement = pd.read_pickle(path2)
subscribers = pd.read_pickle(path3)
ads = pd.read_excel(path4, skiprows=2)

In [3]:
# Check minimun account creation date
subscribers.account_creation_date.min()

Timestamp('2019-06-30 00:00:19')

In [4]:
# Check maximun account creation date
subscribers.account_creation_date.max()

Timestamp('2020-03-27 23:59:06')

### (All) Calculate the number of users attributed to each channel

In [5]:
# Assign tiers to all customers 
tier1_all = customer[customer.account_creation_date <= '2019-07-30']

tier2_all = customer[(customer.account_creation_date > '2019-07-30') & 
                    (customer.account_creation_date <= '2019-08-30')]

tier3_all = customer[(customer.account_creation_date > '2019-08-30') & 
                    (customer.account_creation_date <= '2019-09-30')]

tier4_all= customer[(customer.account_creation_date > '2019-09-30') & 
                    (customer.account_creation_date <= '2019-10-30')]

tier5_all = customer[(customer.account_creation_date > '2019-10-30') & 
                    (customer.account_creation_date <= '2019-11-30')]

tier6_all = customer[(customer.account_creation_date > '2019-11-30') & 
                    (customer.account_creation_date <= '2019-12-30')]

tier7_all = customer[(customer.account_creation_date > '2019-12-30') & 
                    (customer.account_creation_date <= '2020-01-30')]

tier8_all = customer[(customer.account_creation_date > '2020-01-30')& 
                    (customer.account_creation_date <= '2020-02-29')]

tier9_all = customer[(customer.account_creation_date > '2020-02-29') & 
                    (customer.account_creation_date <= '2020-03-30')]

In [6]:
# Calculate total signups for all users 
ads_adjusted = pd.DataFrame()
signups = []
tiers = [tier1_all, tier2_all, tier3_all, tier4_all, tier5_all, tier6_all, tier7_all, tier8_all, tier9_all]

for tier in tiers:
    signup = tier.subid.nunique()
    signups.append(signup)
    
Total_signup = pd.DataFrame(signups,columns=['Total_signup'])
ads_adjusted= pd.concat([ads_adjusted,Total_signup],axis=1)
ads_adjusted.index = list(['tier1', 'tier2', 'tier3', 'tier4', 'tier5', 'tier6', 'tier7', 'tier8', 'tier9'])
ads_adjusted

,Total_signup
tier1,237436
tier2,103644
tier3,114977
tier4,113522
tier5,191968
tier6,133429
tier7,104005
tier8,168230
tier9,202149


### (known) Calculate the number of users attributed to each channel - Technical attribution data is known

In [7]:
# Assign tiers to subscribers data
tier1 = subscribers[subscribers.account_creation_date <= '2019-07-30']

tier2 = subscribers[(subscribers.account_creation_date > '2019-07-30') & 
                    (subscribers.account_creation_date <= '2019-08-30')]

tier3 = subscribers[(subscribers.account_creation_date > '2019-08-30') & 
                    (subscribers.account_creation_date <= '2019-09-30')]

tier4 = subscribers[(subscribers.account_creation_date > '2019-09-30') & 
                    (subscribers.account_creation_date <= '2019-10-30')]

tier5 = subscribers[(subscribers.account_creation_date > '2019-10-30') & 
                    (subscribers.account_creation_date <= '2019-11-30')]

tier6 = subscribers[(subscribers.account_creation_date > '2019-11-30') & 
                    (subscribers.account_creation_date <= '2019-12-30')]

tier7 = subscribers[(subscribers.account_creation_date > '2019-12-30') & 
                    (subscribers.account_creation_date <= '2020-01-30')]

tier8 = subscribers[(subscribers.account_creation_date > '2020-01-30')& 
                    (subscribers.account_creation_date <= '2020-02-29')]

tier9 = subscribers[(subscribers.account_creation_date > '2020-02-29') & 
                    (subscribers.account_creation_date <= '2020-03-30')]

In [8]:
# non-paid: 'internal', 'discovery', 'organic', 'google_organic','facebook_organic', 'bing_organic', 'pinterest_organic'
# Calcualte the number of users attributed to each channel 
channels = ['facebook', 'email', 'search', 'brand sem intent google',
       'affiliate', 'email_blast', 'pinterest', 'referral']
tiers = [tier1, tier2, tier3, tier4, tier5, tier6, tier7, tier8, tier9]
acquisition = pd.DataFrame()

for tier in tiers:
    num_cust = pd.DataFrame(tier['attribution_technical'].value_counts()).T
    acquisition = acquisition.append(num_cust)
    
acquisition.index = list(['tier1', 'tier2', 'tier3', 'tier4', 'tier5', 'tier6', 'tier7', 'tier8', 'tier9'])
acquisition_sub = acquisition[channels]
acquisition_sub

,facebook,email,search,brand sem intent google,affiliate,email_blast,pinterest,referral
tier1,11155,3488,1390,2234,806,1612,800,534
tier2,9427,2924,1781,1787,1867,1844,610,488
tier3,6583,2609,1973,1649,1198,1642,558,604
tier4,9252,2282,2315,1573,759,981,674,523
tier5,12608,2502,2472,1984,505,629,584,594
tier6,9632,2721,3664,2335,550,409,804,660
tier7,6223,2348,4021,2689,1026,37,867,584
tier8,8019,3164,4424,2424,1503,64,676,593
tier9,7352,3652,3266,1849,1680,59,492,590


## Ads spend allocation

In [9]:
# Drop index=9 because we do not have customers who signed up after 3/31/2020
subscribers.account_creation_date.max()
ads = ads.drop(index=9)

In [10]:
# Calculate signups of which technical attribution is known
known_signup = pd.DataFrame(acquisition_sub.sum(axis=1),columns=['Sign ups we know data for'])
ads_adjusted = pd.concat([ads_adjusted,known_signup],axis=1)
ads_adjusted

,Total_signup,Sign ups we know data for
tier1,237436,22019
tier2,103644,20728
tier3,114977,16816
tier4,113522,18359
tier5,191968,21878
tier6,133429,20775
tier7,104005,17795
tier8,168230,20867
tier9,202149,18940


In [11]:
# Get the % of customers we know data for
ads_adjusted['% of customers we know'] = ads_adjusted['Sign ups we know data for'] / ads_adjusted['Total_signup']
ads_adjusted

,Total_signup,Sign ups we know data for,% of customers we know
tier1,237436,22019,0.092737
tier2,103644,20728,0.199992
tier3,114977,16816,0.146255
tier4,113522,18359,0.161722
tier5,191968,21878,0.113967
tier6,133429,20775,0.155701
tier7,104005,17795,0.171098
tier8,168230,20867,0.124039
tier9,202149,18940,0.093693


In [12]:
ads_spend_for_subscribers = ads.copy()
for i in list(range(0,9)):
    for channel in channels:
        ads_spend_for_subscribers[channel][i] = ads_spend_for_subscribers[channel][i] * ads_adjusted['% of customers we know'][i]

C:\Users\hahas\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [13]:
ads_spend_for_subscribers

,date,facebook,email,search,brand sem intent google,affiliate,email_blast,pinterest,referral
0,2019-06-30,7864,6955,1604,1984,1103,973,547,565
1,2019-07-31,11999,15119,3459,4259,2399,5959,1199,1319
2,2019-08-31,7649,11130,2647,3115,2383,9638,906,994
3,2019-09-30,7924,12258,3056,3444,2652,11918,970,1196
4,2019-10-31,5846,8661,2393,2438,1390,3681,763,831
5,2019-11-30,8734,11817,3331,3363,1821,1525,980,1136
6,2019-12-31,8794,13020,5492,3798,2070,598,1231,1368
7,2020-01-31,6077,9364,4167,2815,1773,37,831,917
8,2020-02-29,4656,7176,2820,2033,1405,28,599,609


## CAC by tier by channel for all

In [14]:
# CAC by tier by channel 
dic = {}
CAC_by_tier_by_channel = pd.DataFrame()
channels = ['facebook', 'email', 'search', 'brand sem intent google',
       'affiliate', 'email_blast', 'pinterest', 'referral']
for i in list(range(0,9)):
    for channel in channels:
        CAC = ads_spend_for_subscribers.iloc[i][channel]/acquisition.iloc[i][channel]
        dic[channel] = CAC
    df = pd.DataFrame(dic, index=['tier'+str(i)])
    CAC_by_tier_by_channel = CAC_by_tier_by_channel.append(df) 

CAC_by_tier_by_channel

,facebook,email,search,brand sem intent google,affiliate,email_blast,pinterest,referral
tier0,0.704975,1.993979,1.153957,0.888093,1.368486,0.603598,0.683750,1.058052
tier1,1.272833,5.170657,1.942167,2.383324,1.284949,3.231562,1.965574,2.702869
tier2,1.161932,4.266002,1.341612,1.889024,1.989149,5.869671,1.623656,1.645695
tier3,0.856463,5.371604,1.320086,2.189447,3.494071,12.148828,1.439169,2.286807
tier4,0.463674,3.461631,0.968042,1.228831,2.752475,5.852146,1.306507,1.398990
tier5,0.906769,4.342889,0.909116,1.440257,3.310909,3.728606,1.218905,1.721212
tier6,1.413145,5.545145,1.365829,1.412421,2.017544,16.162162,1.419839,2.342466
tier7,0.757825,2.959545,0.941908,1.161304,1.179641,0.578125,1.229290,1.546374
tier8,0.633297,1.964951,0.863442,1.099513,0.836310,0.474576,1.217480,1.032203


In [15]:
# add mean at the bottom
avg = pd.DataFrame(CAC_by_tier_by_channel.mean()).T
avg.index = ['mean']
CAC_by_tier_by_channel = CAC_by_tier_by_channel.append(avg)

In [16]:
CAC_by_tier_by_channel

,facebook,email,search,brand sem intent google,affiliate,email_blast,pinterest,referral
tier0,0.704975,1.993979,1.153957,0.888093,1.368486,0.603598,0.683750,1.058052
tier1,1.272833,5.170657,1.942167,2.383324,1.284949,3.231562,1.965574,2.702869
tier2,1.161932,4.266002,1.341612,1.889024,1.989149,5.869671,1.623656,1.645695
tier3,0.856463,5.371604,1.320086,2.189447,3.494071,12.148828,1.439169,2.286807
tier4,0.463674,3.461631,0.968042,1.228831,2.752475,5.852146,1.306507,1.398990
tier5,0.906769,4.342889,0.909116,1.440257,3.310909,3.728606,1.218905,1.721212
tier6,1.413145,5.545145,1.365829,1.412421,2.017544,16.162162,1.419839,2.342466
tier7,0.757825,2.959545,0.941908,1.161304,1.179641,0.578125,1.229290,1.546374
tier8,0.633297,1.964951,0.863442,1.099513,0.836310,0.474576,1.217480,1.032203
mean,0.907879,3.897378,1.200684,1.521357,2.025948,5.405475,1.344908,1.748297


## CAC by tier by channel for paid users only

In [17]:
paid = subscribers[subscribers.paid_TF==True]
paid.shape

(92050, 29)

In [18]:
tier1_p = paid[paid.account_creation_date <= '2019-07-30']

tier2_p = paid[(paid.account_creation_date > '2019-07-30') & 
                    (paid.account_creation_date <= '2019-08-30')]

tier3_p = paid[(paid.account_creation_date > '2019-08-30') & 
                    (paid.account_creation_date <= '2019-09-30')]

tier4_p = paid[(paid.account_creation_date > '2019-09-30') & 
                    (paid.account_creation_date <= '2019-10-30')]

tier5_p = paid[(paid.account_creation_date > '2019-10-30') & 
                    (paid.account_creation_date <= '2019-11-30')]

tier6_p = paid[(paid.account_creation_date > '2019-11-30') & 
                    (paid.account_creation_date <= '2019-12-30')]

tier7_p = paid[(paid.account_creation_date > '2019-12-30') & 
                    (paid.account_creation_date <= '2020-01-30')]

tier8_p = paid[(paid.account_creation_date > '2020-01-30')& 
                    (paid.account_creation_date <= '2020-02-29')]

tier9_p = paid[(paid.account_creation_date > '2020-02-29') & 
                    (paid.account_creation_date <= '2020-03-30')]

In [19]:
# Calculating the number of paid users acquired by tier by channel
channels = ['facebook', 'email', 'search', 'brand sem intent google',
       'affiliate', 'email_blast', 'pinterest', 'referral']

tiers = [tier1_p, tier2_p, tier3_p, tier4_p, tier5_p, tier6_p, tier7_p, tier8_p, tier9_p]
acquisition_p = pd.DataFrame()

for tier in tiers:
    num_cust = pd.DataFrame(tier['attribution_technical'].value_counts()).T
    acquisition_p = acquisition_p.append(num_cust)
    
acquisition_p.index = list(['tier1', 'tier2', 'tier3', 'tier4', 'tier5', 'tier6', 'tier7', 'tier8', 'tier9'])
acquisition_sub_p = acquisition_p[channels]
acquisition_sub_p

,facebook,email,search,brand sem intent google,affiliate,email_blast,pinterest,referral
tier1,4150,1559,453,928,382,844,233,277
tier2,3402,1235,645,824,404,890,223,287
tier3,2357,1089,698,767,496,843,194,340
tier4,3225,962,799,745,229,505,223,297
tier5,4774,1068,938,921,144,343,219,345
tier6,3650,1107,1351,1013,223,212,284,375
tier7,2367,1002,1581,1208,389,23,320,354
tier8,3166,1245,1681,1061,518,36,268,327
tier9,3014,1386,1250,800,593,28,189,351


In [20]:
# CAC_by_tier_by_channel for paid users 
dic = {}
CAC_by_tier_by_channel_paid = pd.DataFrame()
channels = ['facebook', 'email', 'search', 'brand sem intent google',
       'affiliate', 'email_blast', 'pinterest', 'referral']
for i in list(range(0,9)):
    for channel in channels:
        CAC = ads_spend_for_subscribers.iloc[i][channel]/acquisition_p.iloc[i][channel]
        dic[channel] = CAC
    df = pd.DataFrame(dic, index=['tier'+str(i)])
    CAC_by_tier_by_channel_paid = CAC_by_tier_by_channel_paid.append(df) 

CAC_by_tier_by_channel_paid

,facebook,email,search,brand sem intent google,affiliate,email_blast,pinterest,referral
tier0,1.894940,4.461193,3.540839,2.137931,2.887435,1.152844,2.347639,2.039711
tier1,3.527043,12.242105,5.362791,5.168689,5.938119,6.695506,5.376682,4.595819
tier2,3.245227,10.220386,3.792264,4.061278,4.804435,11.432977,4.670103,2.923529
tier3,2.457054,12.742204,3.824781,4.622819,11.580786,23.600000,4.349776,4.026936
tier4,1.224550,8.109551,2.551173,2.647123,9.652778,10.731778,3.484018,2.408696
tier5,2.392877,10.674797,2.465581,3.319842,8.165919,7.193396,3.450704,3.029333
tier6,3.715251,12.994012,3.473751,3.144040,5.321337,26.000000,3.846875,3.864407
tier7,1.919457,7.521285,2.478882,2.653157,3.422780,1.027778,3.100746,2.804281
tier8,1.544791,5.177489,2.256000,2.541250,2.369309,1.000000,3.169312,1.735043


In [21]:
# add mean at the bottem
avg = pd.DataFrame(CAC_by_tier_by_channel_paid.mean()).T
avg.index = ['mean']
CAC_by_tier_by_channel_paid = CAC_by_tier_by_channel_paid.append(avg)

In [22]:
CAC_by_tier_by_channel_paid

,facebook,email,search,brand sem intent google,affiliate,email_blast,pinterest,referral
tier0,1.894940,4.461193,3.540839,2.137931,2.887435,1.152844,2.347639,2.039711
tier1,3.527043,12.242105,5.362791,5.168689,5.938119,6.695506,5.376682,4.595819
tier2,3.245227,10.220386,3.792264,4.061278,4.804435,11.432977,4.670103,2.923529
tier3,2.457054,12.742204,3.824781,4.622819,11.580786,23.600000,4.349776,4.026936
tier4,1.224550,8.109551,2.551173,2.647123,9.652778,10.731778,3.484018,2.408696
tier5,2.392877,10.674797,2.465581,3.319842,8.165919,7.193396,3.450704,3.029333
tier6,3.715251,12.994012,3.473751,3.144040,5.321337,26.000000,3.846875,3.864407
tier7,1.919457,7.521285,2.478882,2.653157,3.422780,1.027778,3.100746,2.804281
tier8,1.544791,5.177489,2.256000,2.541250,2.369309,1.000000,3.169312,1.735043
mean,2.435688,9.349225,3.305118,3.366237,6.015877,9.870475,3.755095,3.047528


## Exports files

In [23]:
CAC_by_tier_by_channel.to_csv('CAC_by_tier_by_channel.csv',index=False)

In [24]:
CAC_by_tier_by_channel_paid.to_csv('CAC_by_tier_by_channel_paid.csv',index=False)